In [9]:
# Importing CSV files and display them as dataframes
import numpy as np
import pandas as pd
rating_dataset = pd.read_csv("ratings.csv")
rating_dataset = rating_dataset.drop(columns =['timestamp'])
rating_dataset.head()
movie_dataset = pd.read_csv("movies.csv")
#movie_dataset.head()
data = pd.merge(rating_dataset,movie_dataset, left_on='movieId',right_on ='movieId',how='left')
data= data

data.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [10]:
# check if there is any data in any column missing or not available
data.isnull().values.any()

False

In [11]:
# find threshold by averging the movie ratings
# get only top5 rated movies per user,threshold is 4.5
maxRating = data.loc[data['rating'].idxmax()]['rating'] 
toprated_user_movies = data.loc[data['rating'] >= maxRating/2+1]
toprated_user_movies = toprated_user_movies.sort_values(['userId','rating'],ascending=[True,False])
toprated_user_movies.reset_index(inplace=True, drop=True)
toprated_user_movies.head()

# checking if user has rated a movie multiple times
#toprated_user_movies.duplicated(subset=['userId','movieId'])
# actual data is 20000263 after this step it is 646297

,userId,movieId,rating,title,genres
0,1,4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,1,5952,5.0,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
2,1,7153,5.0,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
3,1,8507,5.0,Freaks (1932),Crime|Drama|Horror
4,1,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi


In [12]:
import pandas as pd
unique_genres_pre = pd.DataFrame(columns=['genres'])
#unique_genres_pre['genres'] = unique_genres_pre['genres'].str.split('|')
unique_genres_pre['genres'] = toprated_user_movies['genres'].unique()
unique_genres_pre['genres'] = unique_genres_pre['genres'].str.split('|')
unique_genres = unique_genres_pre['genres']

In [13]:
# Break up the big genre string into a string array
toprated_user_movies['genres'] = toprated_user_movies['genres'].str.split('|')
# Convert genres to string value
#toprated_user_movies['genres'] = np.array(toprated_user_movies['genres'])
toprated_user_movies['genres'] = toprated_user_movies['genres'].str.join(',')
toprated_user_movies.head()

,userId,movieId,rating,title,genres
0,1,4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...","Adventure,Fantasy"
1,1,5952,5.0,"Lord of the Rings: The Two Towers, The (2002)","Adventure,Fantasy"
2,1,7153,5.0,"Lord of the Rings: The Return of the King, The...","Action,Adventure,Drama,Fantasy"
3,1,8507,5.0,Freaks (1932),"Crime,Drama,Horror"
4,1,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,"Action,Adventure,Sci-Fi"


In [14]:
#target domain
books = pd.read_csv('book_data.csv')
books = books[['book_title','genres','book_rating']]
books = books.dropna(how='any', axis=0)
# Break up the big genre string into a string array
books['genres'] = books['genres'].str.split('|')
# Convert genres to string value
books['genres'] = np.array(books['genres'])

books['length'] = books['genres'].str.len()
books = books.sort_values(['length'])
books = books.drop(columns = ['length'])
books = books.loc[books['book_rating'] >= 3]
books.reset_index(inplace=True, drop=True)
books.head()

,book_title,genres,book_rating
0,The Caine Batter,[Literature],3.20
1,الأيام,[Biography],3.53
2,Biological Big Bang: Panspermia and the Origin...,[Nonfiction],4.50
3,Mugglenet.Com's What Will Happen in Harry Pott...,[Nonfiction],4.20
4,"Crónica del rey cautivo (El último Qassatar, #1)",[Fantasy],4.13


In [15]:
def book_basedon_movie(genre):
    # finding intersecti:on and recommending books
    data = []
   
    book_titles = []
    scores = []
    ratings = []
    for i,row in books.iterrows():
        movies_set = set(genre)
        books_set = set(row.genres)
        intersection = movies_set.intersection(books_set)
        book_titles.append(row.book_title)
        ratings.append(row.book_rating)
        scores.append(float(len(intersection))/(len(movies_set)+len(books_set)))
    final = pd.DataFrame(columns = ['book_title','score','ratings'])
    final['book_title'] = book_titles
    final['score'] = scores
    final['ratings'] = ratings
    return final



#toprated_user_movies['books'] = books_array

#toprated_user_movies.head()




In [ ]:
df_books_movies = pd.DataFrame(columns=['genres','books'])
size = len(unique_genres)
books_array = []
genres = []
for i in range(size):
    book_df = book_basedon_movie(unique_genres[i])
    #book_df = book_df.loc[book_df['ratings'] >= 3]
    book_df = book_df.sort_values(['score','ratings'],ascending=[False,False]).head(5)
    books_array.append(','.join([str(elem) for elem in book_df['book_title']]))
    genres.append(','.join(unique_genres[i]))
df_books_movies['genres'] = genres
df_books_movies['books'] = books_array
    
df_books_movies.head()

In [143]:
final = pd.merge(toprated_user_movies,df_books_movies, left_on='genres',right_on ='genres',how='left')
final.head()

,userId,movieId,rating,title,genres,books
0,1,4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...","Adventure,Fantasy",NaN
1,1,5952,5.0,"Lord of the Rings: The Two Towers, The (2002)","Adventure,Fantasy",NaN
2,1,7153,5.0,"Lord of the Rings: The Return of the King, The...","Action,Adventure,Drama,Fantasy",NaN
3,1,8507,5.0,Freaks (1932),"Crime,Drama,Horror",NaN
4,1,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,"Action,Adventure,Sci-Fi",NaN


In [145]:
#testing
print df_books_movies.loc[df_books_movies['genres']=='Adventure,Children,Fantasy']

                       genres  \
0  Adventure,Children,Fantasy   

                                               books  
0  Selections from Harry Potter and the Order of ...  
